In [1]:
import os
import sys
import json
import csv
import glob
import logging
import unittest
from datetime import datetime, timezone
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Any
import boto3
import numpy as np
import pystac
import rasterio
from osgeo import gdal
from botocore.config import Config
from botocore.exceptions import ClientError
from pystac.asset import Asset
from pystac.errors import ExtensionTypeError, RequiredPropertyMissing, STACError
from pystac.extensions.pointcloud import (
    Schema, SchemaType, AssetPointcloudExtension, PhenomenologyType, 
    PointcloudExtension, Statistic
)
from pystac.summaries import RangeSummary
from rasterio.plot import reshape_as_image, reshape_as_raster
from rasterio.transform import from_bounds
from rio_cogeo import cog_validate
from shapely.geometry import Polygon, mapping, shape
from pyproj import Transformer
import constants
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import re



config = Config(signature_version='s3v4')

#set up logging 
logging.basicConfig(level=logging.DEBUG)
debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

loading dot env...


# NEED TO MAKE SOMETHING THAT CHECKS FOR EXISTING CATALOG IF YOU ARE ADDING TO

In [2]:
#variables 
# prefixes for saved stac jsons

output_location=r'/mnt/w_drive/srm/gss/sandbox/cfolkers/stac/output/'

json_out=fr"{output_location}jsons"
thumbnail_out=fr"{output_location}/thumbnails"

# Object Storage where point clouds are stored 
catalog_name='BC_STAC'
collection_name='DEM'
public_asset_loc=r'https://nrs.objectstore.gov.bc.ca/gdwuts'
obj_type=r'DEM' #the type of asset DEM, DSM, CRH, Landcover ect
file_extension='.tif'
object_store=r"/mnt/r_drive"
map_area="092"
subdivision='o'
# tiles=[18] #rest of test area with DEMS 19,20,28,29,30
root_directory = Path(fr"{object_store}/{map_area}/{map_area}{subdivision}")
# location for STAC jsons in s3, used to create permanent s3 links and the base link for all collections and items from this script
collection_loc=rf"{catalog_name}/{obj_type}"
json_out_dir=Path(r"/home/cfolkers/STAC_LiDAR/PointClouds/json/")
invalid_output=r'invalid_cogs.csv'

In [3]:
#S3 access 


# # use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)
# for some reason the bucket is adding an extra letter at the end???
bucket = constants.AWS_S3_BUCKET

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [4]:
#function to get all bounding boxes and footprints from rasters
def get_bbox_footprint_and_metadata(raster):
    file_size_kb=os.path.getsize(raster) >>10
    with rasterio.open(raster) as r:
        # Get the bounds of the raster
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        transformer = Transformer.from_crs(r.crs, "EPSG:4326", always_xy=True)
        ll_lon, ll_lat = transformer.transform(bbox[0], bbox[1])  # lower-left corner
        ur_lon, ur_lat = transformer.transform(bbox[2], bbox[3])  # upper-right corner
        print('Transformed lower-left:', (ll_lon, ll_lat))
        print('Transformed upper-right:', (ur_lon, ur_lat))
        footprint = Polygon([
            [ll_lon, ll_lat],  # lower-left
            [ll_lon, ur_lat],  # upper-left
            [ur_lon, ur_lat],  # upper-right
            [ur_lon, ll_lat]   # lower-right
        ])
        
        metadata = r.meta
        metadata['crs'] = metadata['crs'].to_epsg()
        
        gt = r.transform
        pixelSizeX = gt[0]
        pixelSizeY =-gt[4]
        resolution= (pixelSizeX, pixelSizeY, r.crs.linear_units)
        
        validate=cog_validate(raster)
        if validate[0] == True:
            filetype='COG'
            metadata['driver']=filetype
        metadata['resolution']=resolution
        metadata['file size']=f"{file_size_kb} KB"
        
        
        # crs= r.crs
        # metadata.update({'crs':crs})

        return ([ll_lon, ll_lat, ur_lon, ur_lat], mapping(footprint), metadata, )
    
    
def create_url(bucket_name: str,
            object_name: str):
    """
           
    This function takes a bucket name, an object name, and an expiration time (in seconds) and generates a URL download link for the object.

    Arguments:
        bucket_name: String of name of the bucket
        object_name: Name of the object (key) that the URL will be pointed to

    Returns:
        Link of output (object download) URL
        
    Raises: 
        Exceptions raised will display an error message and be logged in the export.log file
    """
    try:
        if r':443' in constants.AWS_S3_ENDPOINT:
            endpoint=constants.AWS_S3_ENDPOINT.split(':')
            endpoint=fr"{endpoint[0]}:{endpoint[1]}"
        else:
            endpoint=constants.AWS_S3_ENDPOINT
        response=os.path.join(endpoint,bucket_name,object_name)
    except ClientError as e:
        print(e)
        return None
    return response

def set_permissions(bucket_name: str,
                    object_name: str,
                    permissions='public-read'):
    """
    This function takes a bucket name, an object name, and a permissions value (specified below) and sets the object's permissions to the value given.

    Arguments:
        bucket_name: String of name of the bucket
        object_name: Name of the object (key) that the URL will be pointed to
        permissions: If not specified, the permissions will default to 'public-read'. Otherwise, permissions can be found below:
        'private'|'public-read'|'public-read-write'|'authenticated-read'|'aws-exec-read'|'bucket-owner-read'|'bucket-owner-full-control'

    Returns:
        Nothing
        
    Raises: 
        Exceptions raised will display an error message and be logged in the export.log file 
    """

    try:
        response = s3_client.put_object_acl(ACL=permissions, Bucket=bucket_name, Key=object_name)
        print(f'Set permissions on {object_name} success, set to {permissions}')
    except Exception as e:
        print(f'Error when setting permission: double check permission: {permissions}. Refer to help(set_permissions) for documentation.')
        print(e)

    return

def write_list_to_csv(data_list, filename):
    """Writes a list to a CSV file, each item in a new row."""
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        for item in data_list:
            writer.writerow([item])  # Writing each item in a new row

def create_thumbnail(dem_path, thumbnail_path, max_size=256):
    with rasterio.open(dem_path) as src:
        dem = src.read(1) 

        nodata = src.nodata
        if nodata is not None:
            dem = np.where(dem == nodata, np.nan, dem)  

        vmin, vmax = np.nanmin(dem), np.nanmax(dem)
        dem_scaled = (dem - vmin) / (vmax - vmin) 

        height, width = dem.shape
        aspect_ratio = width / height

        if width > height:
            new_width = max_size
            new_height = int(max_size / aspect_ratio)
        else:
            new_height = max_size
            new_width = int(max_size * aspect_ratio)
            
        zoom_factor_x = new_height / height
        zoom_factor_y = new_width / width
        dem_resized = zoom(dem_scaled, (zoom_factor_x, zoom_factor_y), order=1)  # Bilinear interpolation

    fig, ax = plt.subplots(figsize=(new_width / 100, new_height / 100), dpi=100)
    ax.imshow(dem_resized, cmap='gray', interpolation='bilinear', alpha=1)  # Adjust alpha if needed
    ax.set_axis_off()

    fig.savefig(thumbnail_path, bbox_inches='tight', pad_inches=0, transparent=True, dpi=100)
    plt.close(fig)

def extract_dates(file_path):
    file_path = str(file_path)
    year_match = re.search(r'/(\d{4})/', file_path)
    year = year_match.group(1) if year_match else None
    date_match = re.findall(r'_(\d{8})', file_path)
    
    if len(date_match) == 2:
        date_start = datetime.strptime(date_match[0], "%Y%m%d")
        date_end = datetime.strptime(date_match[1], "%Y%m%d")
    elif len(date_match) == 1:
        date_start = datetime.strptime(date_match[0], "%Y%m%d")
        date_end = date_start
    elif year:
        date_start = datetime.strptime(year, "%Y")
        date_end = date_start
    else:
        date_start, date_end = None, None

    return date_start, date_end

In [5]:
#create base url for all json hrefs 
base_url=create_url(bucket, collection_loc)
print(base_url)



https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/DEM


In [6]:
#find and filter raster files and get file path
rast_files=list(Path(root_directory).rglob(f"{obj_type.lower()}/*{file_extension}"))
logging.info(f"{len(rast_files)} {file_extension} files found")

#can be used to filter to specific tiles if needed 
# filtered_files = []
# for rast in rast_files:
#     stem = rast.stem
#     tile_str = stem[7:10]
#     try:
#         tile_num = int(tile_str)
#         if tile_num in tiles:
#             filtered_files.append(filtered_files)
#     except ValueError:
#         continue
# logging.info(f"Filtered down to {len(filtered_files)} files whose tiles are in {tiles}.")

INFO:root:43 .tif files found


In [7]:
# create STAC Catalog 
#https://pystac.readthedocs.io/en/stable/api/catalog.html

url=create_url(bucket, os.path.join(catalog_name,f"{catalog_name}_catalog.json"))

catalog = pystac.Catalog(id=catalog_name,
                        description='This catalog is contains Open Data from the Province of BC',
                        stac_extensions=None, #if not none list of extension
                        extra_fields = None, #if not none dictionary
                        href=url,
                        catalog_type='ABSOLUTE_PUBLISHED', # https://pystac.readthedocs.io/en/stable/api/pystac.html#pystac.CatalogType
                        strategy=None) # https://pystac.readthedocs.io/en/stable/api/layout.html#pystac.layout.BestPracticesLayoutStrategy    https://github.com/radiantearth/stac-spec/blob/v1.1.0/best-practices.md#catalog-layout
                        

print(json.dumps(catalog.to_dict(), indent=4))


print(F"THIS IS THE URL {url}   !!!!!!!!!!!!!!!!!!!!!!!!")

{
    "type": "Catalog",
    "id": "BC_STAC",
    "stac_version": "1.1.0",
    "description": "This catalog is contains Open Data from the Province of BC",
    "links": [
        {
            "rel": "root",
            "href": "https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/BC_STAC_catalog.json",
            "type": "application/json"
        },
        {
            "rel": "self",
            "href": "https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/BC_STAC_catalog.json",
            "type": "application/json"
        }
    ]
}
THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/BC_STAC_catalog.json   !!!!!!!!!!!!!!!!!!!!!!!!


In [8]:
invalid_cogs=[]
item_href_list=[]

for filename in rast_files:
    # validate=cog_validate(filename)
    # if validate[0] == True:
    # info("Valid COG")
    
    asset_suffix=str(filename).split('drive')[-1]
    asset_suffix=asset_suffix.split('.tif')[0]
    
    asset_file=fr"{public_asset_loc}{asset_suffix}"
    item_name=str(filename).split('/')[-1]
    item_name=item_name.split('.tif')[0]
    thumbnail_output=f"{thumbnail_out}/{item_name}.png"
    thumbnail_file=f"{asset_file}.png"

    # info(asset_suffix)
    # info(asset_file)
    # info(item_name)
    item_href=f"{base_url}/{item_name}/{item_name}.json"
    thumbnail_file=f"{base_url}/{item_name}/{item_name}.png"
    
    item_href_list.append(item_href)          
    info(f"Item Href is : {item_href}")        
    bbox, footprint, meta = get_bbox_footprint_and_metadata(filename)
    thumbnail=create_thumbnail(filename, thumbnail_output)
    # info("bbox: %s", bbox)
    # info("footprint: ", footprint)
    
    date_start, date_end =extract_dates(filename)
    datetime_utc = datetime.now(tz=timezone.utc)

    
    #create items
    #https://pystac.readthedocs.io/en/stable/api/item.html
    
    item = pystac.Item(id=item_name,
                    geometry=footprint,
                    bbox=bbox,
                    datetime=date_start, #if none a start and end datetime must be supplied 
                    properties=meta, #additional metadata for the item
                    start_datetime=date_start,
                    end_datetime=date_end,
                    href= item_href
                    )
    #add link to data as asset in this case direct link to the DEM to be accessed
    item.add_asset(
    key=f"{item_name}_data",
    asset=pystac.Asset(
        href=asset_file,
        media_type=pystac.MediaType.COG,
        roles=['data']
        )
    )
    #add thumbnail
    item.add_asset(
    key=f"{item_name}_thumbnail",
    asset=pystac.Asset(
        href=thumbnail_file,
        media_type=pystac.MediaType.PNG,
        roles=['thumbnail','overview','visual']
        )
    )
    # pystac.Item.set_self_href(item_href)
    catalog.add_item(item)
    item.set_self_href(item_href)

    
    # else:
    #     print("file not a valid COG, need to covert check output list")
    #     invalid_cogs.append(filename)
   
    
    
# #write invalid cog list to csv 
# if invalid_cogs is not None:
#     write_list_to_csv(invalid_cogs, invalid_output)


INFO:root:Item Href is : https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/DEM/bc_092o009_xli1m_utm10_2018/bc_092o009_xli1m_utm10_2018.json
DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7f2e53cfc6e0>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7f2e7602cc10> created
DEBUG:rasterio._filepath:Installing FilePath filesystem handler plugin...
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_DATA found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f2e7602cc10>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7f2e53cfc6e0>
DEBUG:rasterio._base:Sharing flag: 0
DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='/mnt/r_drive/092/092o/2018/dem/bc_092o009_xli1m_utm10_2018.tif' mode='r

Transformed lower-left: (-122.36000265959397, 51.060338613556475)
Transformed upper-right: (-122.19968056529491, 51.100015200186924)


DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='/mnt/r_drive/092/092o/2018/dem/bc_092o009_xli1m_utm10_2018.tif' mode='r'> is started.
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f2e538e0170>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f2e53d91c90> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f2e53d91c90>.
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7f2e553ed690> created
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_DATA found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f2e553ed690>.
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f2e538e0170>
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f2e53ce2fd0>
DEBUG:rasterio.env:Cle

Transformed lower-left: (-122.20033799918633, 51.08177382677759)
Transformed upper-right: (-122.11548839464204, 51.09993015768656)


DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='/mnt/r_drive/092/092o/2018/dem/bc_092o010_xli1m_utm10_2018 (2).tif' mode='r'> is started.
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f2e539c9fd0>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f2e75d56a70> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f2e75d56a70>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f2e539c9fd0>
DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-para

...

DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='/mnt/r_drive/092/092o/2023/dem/bc_092o091_xli1m_utm10_20230918_20231006.tif' mode='r'> is started.
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f2e5157de10>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f2e51e4db70> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f2e51e4db70>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f2e5157de10>
DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from be

Transformed lower-left: (-123.80072389732584, 51.898360641872905)
Transformed upper-right: (-123.59930781441194, 52.001653204957265)


DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='/mnt/r_drive/092/092o/2023/dem/bc_092o092_xli1m_utm10_20230918_20231006.tif' mode='r'> is started.
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f2e5168e890>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f2e51f57190> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f2e51f57190>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f2e5168e890>
DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from be

Transformed lower-left: (-123.60072745436824, 51.89869726432093)
Transformed upper-right: (-123.39929715464862, 52.001314162627075)


DEBUG:rasterio._base:Nodata success: 1, Nodata value: -32767.000000
DEBUG:rasterio._base:Dataset <open DatasetReader name='/mnt/r_drive/092/092o/2023/dem/bc_092o093_xli1m_utm10_20230918_20231006.tif' mode='r'> is started.
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f2e5168f770>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f2e51fd19c0> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f2e51fd19c0>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f2e5168f770>
DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from be

In [9]:
item

<Item id=bc_092o093_xli1m_utm10_20230918_20231006>

In [10]:
#get collection bounds
unioned_footprint = None
datetime_list=[]
item_list=[]
for item in catalog.get_all_items():
    datetime_list.append(item.datetime)
    item_list.append(item)
    footprint = item.geometry
# Convert the footprint geometry to a Shapely shape
    footprint_shape = shape(footprint)
    
    # Perform union operation
    if unioned_footprint is None:
        # If unioned_footprint is None (first iteration), initialize it with the first footprint
        unioned_footprint = footprint_shape
    else:
        # Otherwise, perform union with the current footprint
        unioned_footprint = unioned_footprint.union(footprint_shape)

collection_bbox = list(unioned_footprint.bounds)
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

In [11]:
filtered_list = [dt for dt in datetime_list if dt is not None]

In [12]:
collection_interval = sorted(filtered_list)
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [13]:


collection_href= f"{collection_loc}/collection.json"
url=create_url(bucket, collection_href)
collection = pystac.Collection(id=collection_name,
                            description='Digital Elevation Models for British Columbia',
                            extent=collection_extent,
                            title=collection_name,
                            href=url,
                            license='Apache-2.0',
                            keywords=('Digital Elevation Model','DEM','Raster','LiDAR'))
collection.set_self_href(url)


print(F"THIS IS THE URL {url}   !!!!!!!!!!!!!!!!!!!!!!!!")

THIS IS THE URL https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/DEM/collection.json   !!!!!!!!!!!!!!!!!!!!!!!!


In [14]:
collection.add_items(item_list)
catalog.add_child(collection)

<Link rel=child target=<Collection id=DEM>>

In [15]:
#view parent data
item.get_parent()

<Collection id=DEM>

In [16]:
#visualize STAC Architecture, be careful when using it on large catalogs, as it will walk the entire tree of the STAC.
catalog.describe()

* <Catalog id=BC_STAC>
    * <Collection id=DEM>
      * <Item id=bc_092o009_xli1m_utm10_2018>
      * <Item id=bc_092o010_xli1m_utm10_2018 (2)>
      * <Item id=bc_092o010_xli1m_utm10_2018>
      * <Item id=bc_092o018_xli1m_utm10_2018 (2)>
      * <Item id=bc_092o018_xli1m_utm10_2018>
      * <Item id=bc_092o019_xli1m_utm10_2018 (2)>
      * <Item id=bc_092o019_xli1m_utm10_2018>
      * <Item id=bc_092o020_xli1m_utm10_2018 (2)>
      * <Item id=bc_092o020_xli1m_utm10_2018>
      * <Item id=bc_092o028_xli1m_utm10_2018>
      * <Item id=bc_092o029_xli1m_utm10_2018>
      * <Item id=bc_092o030_xli1m_utm10_2018>
      * <Item id=bc_092o007_xli1m_utm10_2019>
      * <Item id=bc_092o008_xli1m_utm10_2019>
      * <Item id=bc_092o009_xli1m_utm10_2019>
      * <Item id=bc_092o017_xli1m_utm10_2019>
      * <Item id=bc_092o020_xli1m_utm10_2019>
      * <Item id=bc_092o029_xli1m_utm10_2019>
      * <Item id=bc_092o030_xli1m_utm10_2019>
      * <Item id=bc_092o039_xli1m_utm10_2019>
      * <Item i

In [17]:
item

<Item id=bc_092o093_xli1m_utm10_20230918_20231006>

In [18]:
info(json.dumps(item.to_dict(), indent=4))

INFO:root:{
    "type": "Feature",
    "stac_version": "1.1.0",
    "stac_extensions": [],
    "id": "bc_092o093_xli1m_utm10_20230918_20231006",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    -123.60072745436824,
                    51.89869726432093
                ],
                [
                    -123.60072745436824,
                    52.001314162627075
                ],
                [
                    -123.39929715464862,
                    52.001314162627075
                ],
                [
                    -123.39929715464862,
                    51.89869726432093
                ],
                [
                    -123.60072745436824,
                    51.89869726432093
                ]
            ]
        ]
    },
    "bbox": [
        -123.60072745436824,
        51.89869726432093,
        -123.39929715464862,
        52.001314162627075
    ],
    "properties": {
      

In [19]:
print("Catalog HREF: ", catalog.get_self_href())
print("Collection HREF:", collection.get_self_href())
print("Item HREF: ", item.get_self_href())

Catalog HREF:  https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/BC_STAC_catalog.json
Collection HREF: https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/DEM/collection.json
Item HREF:  https://nrs.objectstore.gov.bc.ca/cloudgistest/BC_STAC/DEM/bc_092o093_xli1m_utm10_20230918_20231006/bc_092o093_xli1m_utm10_20230918_20231006.json


In [20]:

catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED, dest_href=json_out)